In [26]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics
from calitp_data_analysis import calitp_color_palette

# from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

import altair as alt

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [13]:
## parameters cell
itp_id = 87

In [14]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt

In [15]:
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet')

In [16]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [17]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [18]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [19]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [20]:
#  TODO to utils
def time_period_and_arrowize(gdf, time_period):
    
    gdf = gdf >> filter(_.time_of_day == time_period)
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [21]:
peak_test = time_period_and_arrowize(speedmap_segs, 'AM Peak')

In [22]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [23]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
570445,cf0f7df88da36cd9ca4248eb1d6a0f39,e1bff90163e0598466fb41083a3088c6,shp-6-14,6,1.0,598__599,Sepulveda Blvd/Washington Pl__Sepulveda Blvd/V...,598-599-1,AM Peak,6.66,...,6.66,Culver City Schedule,07 - Los Angeles,rec5ome04BbA9uf4y,City of Culver City,aHR0cHM6Ly93d3cuY3VsdmVyY2l0eS5vcmcvZmlsZXMvYX...,"POLYGON ((146274.764 -443604.993, 146274.090 -...",1,0.333333,CC 6
570446,cf0f7df88da36cd9ca4248eb1d6a0f39,e1bff90163e0598466fb41083a3088c6,shp-6-14,6,1.0,599__600,Sepulveda Blvd/Venice Blvd__Sepulveda Blvd/Cha...,599-600-1,AM Peak,11.64,...,11.64,Culver City Schedule,07 - Los Angeles,rec5ome04BbA9uf4y,City of Culver City,aHR0cHM6Ly93d3cuY3VsdmVyY2l0eS5vcmcvZmlsZXMvYX...,"POLYGON ((146229.212 -443533.121, 146228.278 -...",1,0.333333,CC 6


## export map

In [24]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

NameError: name 'time_of_day' is not defined

In [ ]:
#  TODO clone render method from RtFilterMapper, to utils
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

## docs!

In [48]:
domain = cmap.index

In [54]:
range_ = [cmap.rgb_hex_str(i) for i in cmap.index]

In [49]:
range_ = cmap.colors

In [137]:
df = speedmap_segs[['time_of_day', 'p50_mph', 'p20_mph', 'p80_mph']]
df = df >> group_by(_.time_of_day) >> summarize(p50_mph = _.p50_mph.quantile(.5),
                                               p20_mph = _.p20_mph.quantile(.5),
                                               p80_mph = _.p80_mph.quantile(.5),)
df['p50 - p20'] = -(df['p50_mph'] - df['p20_mph'])
df['p80 - p50'] = df['p80_mph'] - df['p50_mph']

In [138]:
error_bars = alt.Chart(df).mark_errorbar(thickness=5, color='gray', opacity=.6).encode(
    y = alt.Y("p50_mph:Q", title='Segment Speed: 20, 50, 80%ile').scale(zero=False),
    yError=("p50 - p20:Q"),
    yError2=("p80 - p50:Q"),
    x = alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
    tooltip=[alt.Tooltip('p20_mph:Q', title="p20 mph"), alt.Tooltip('p50_mph:Q', title="p50 mph"),
            alt.Tooltip('p80_mph:Q', title="p80 mph")]
).properties(width=400)

points = alt.Chart(df).mark_point(filled=True, size = 150, opacity = 1).encode(
    alt.Y("p50_mph:Q"),
    alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl'],
         title='Time of Day'),
    color=alt.Color('p50_mph', title='Median Segment Speed (mph)').scale(domain=domain, range = range_),
    tooltip=[alt.Tooltip('p50_mph:Q', title="p50 mph")],
)

chart = error_bars + points
chart = chart.configure(axis = alt.AxisConfig(labelFontSize=14, titleFontSize=18),
                       legend = alt.LegendConfig(titleFontSize=14, labelFontSize=14, titleLimit=250,
                                                 titleOrient='left', labelOffset=100))
chart

alt.LayerChart(...)

In [122]:
error_bars

Undefined

In [120]:
dir(error_bars.properties())

['__add__',
 '__and__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_transform',
 '_args',
 '_class_is_valid_at_instantiation',
 '_counter',
 '_default_wrapper_classes',
 '_get',
 '_get_name',
 '_kwds',
 '_repr_mimebundle_',
 '_rootschema',
 '_schema',
 '_set_resolve',
 'add_params',
 'add_selection',
 'align',
 'autosize',
 'background',
 'bounds',
 'center',
 'config',
 'configure',
 'configure_arc',
 'configure_area',
 'configure_axis',
 'configure_axisBand',
 'configure_axisBottom',
 'configure_axisDiscrete',
 'configure_axisLeft',
 'configure_axisPoint',
 'configure_axisQuantit